### Access Relation

访问关系映射实例元素设置为该语句访问的数据元素. 在读取和写入访问之间做出区分通常很重要

- Read Access Relation

  读取访问关系将每个动态执行实例映射到通过动态执行实例读取的数据元素集.

- Write Access Relation

  写入访问关系将每个动态执行实例映射到动态执行实例写入的数据元素集

在某些情况下,无法确定访问的数据元素的确切集. 或者即使有可能确定确切的访问关系,也可能无法将它们表示为presburger relation. 因此需要近似访问关系. 如果读取, 则足以确定访问已访问的数据元素的高估集合. 写入的情况需要更多考虑写入访问关系的某些用途, 例如计算程序片段可能访问的元素集, 这允许高估. 而写入访问关系的其他一些用途不允许高估,而是需要低估. 因此出现以下三种类型的访问关系:

- May-Read Access Relation

  包含了读取访问关系作为子集.


- May-Write Access Relation

  包含了写访问关系作为子集.

- Must-Write Access Relation

  是一个写访问关系的子集.

在编译时无法确切计算的访问关系的情况下, 这种灵活性是有用的, 如果给定的动态执行实例将访问哪些元素, 或者如果无法准确表示此信息. 在此信息可用并且可以准确表示的情况下, 可以将访问关系限制/扩展为包括已访问的数据元素, 比如may-write关系可以等同于must-write关系. 这三个关系都可以以presburger relation的方式表示, 同时对于标量的索引用空tuple来表示. 

In [1]:
from common import isl, pet, CSource, CodeGenerator, Callable, parse_code

In [2]:
scop = parse_code("""
float inner(int n, float A[const restrict static n], float B[const restrict static n]) {
#pragma scop  
   float prod;
S: prod = 0.0;
L: for (int i = 0; i < n; ++i) 
T:  prod += A[i] * B[i];
#pragma endscop
 return prod; 
}
""", "inner")

我们解析函数后获取三种访问关系:

In [3]:
scop.get_may_reads()

isl.union_map("[n] -> { T[i] -> prod[] : 0 <= i < n; T[i] -> A[i] : 0 <= i < n; T[i] -> B[i] : 0 <= i < n }")

In [4]:
scop.get_may_writes()

isl.union_map("[n] -> { S[] -> prod[]; T[i] -> prod[] : 0 <= i < n }")

In [5]:
scop.get_must_writes()

isl.union_map("[n] -> { S[] -> prod[]; T[i] -> prod[] : 0 <= i < n }")

### Tagged Access Relations

标准访问关系是通过该语句实例访问的数据元素的映射的. 但是给定的statement可以多次引用相同的数据结构, 在某些情况下需要区分各个引用. 例如, 当PPCG确定要到/从设备复制的数据时, 它会检查哪些写入引用产生仅在给定内核内使用的数据. 这要求引用可以从依赖关系中识别, 同时需要它们在访问关系中添加进行不同的标记来区分.

在Pet中, 为程序片段中的每个引用生成了唯一的标识符. 然后, 这些标识符用于“标记”语句实例, 在所谓的标记访问关系中执行访问. 特别是, 这种标记的访问关系的域是与语句实例和范围作为引用标识符的包装关系. 可以通过计算域产品域因子来从这种标记的访问关系中删除标签. 标记的访问关系可以使用以下功能从scop中获取:

In [6]:
scop.get_tagged_may_reads()

isl.union_map("[n] -> { [T[i] -> __pet_ref_4[]] -> B[i] : 0 <= i < n; [T[i] -> __pet_ref_3[]] -> A[i] : 0 <= i < n; [T[i] -> __pet_ref_2[]] -> prod[] : 0 <= i < n }")

In [7]:
scop.get_tagged_may_writes()

isl.union_map("[n] -> { [T[i] -> __pet_ref_2[]] -> prod[] : 0 <= i < n; [S[] -> __pet_ref_1[]] -> prod[] }")

In [8]:
scop.get_tagged_must_writes()

isl.union_map("[n] -> { [T[i] -> __pet_ref_2[]] -> prod[] : 0 <= i < n; [S[] -> __pet_ref_1[]] -> prod[] }")

## Dependence Relation

依赖关系是实例集元素之间的二元关系, 其中一个实例以某种方式依赖于另一个实例. 可以考虑几种类型的依赖关系, 一个实例对另一个实例的依赖性的确切性质取决于依赖关系的类型. 但是通常依赖关系表示一个实例需要在另一个实例之前执行. 依赖性的原因通常是两个语句实例涉及访问相同的内存元素. 可以根据所涉及的两个访问的类型来区分不同类型的依赖.

🟢 Read-after-Write Dependence Relation
  
  如果实例j在实例i之后执行, 并且读取被实例i写过的数据点, 那么写后读依赖实例i映射实例j.

🟢 Write-after-Read Dependence Relatio

  如果实例j在实例i之后执行, 并且写入被实例i读过的数据点, 那么读后写依赖实例i映射实例j.

🟢 Write-after-Write Dependence Relation

  如果实例j在实例i之后执行, 并且写入被实例i写入的数据点, 那么写后写依赖实例i映射实例j.


对于这些依赖关系, 同样可有可能无法被精确计算或者被presburger relation所表示. 但是如果只考虑表示执行顺序的先后时, 这些关系可以看作是被高估的. 在后续的教程总所有提到的Read-after-Write依赖, 都是may Read-after-Write依赖, 其他两个也同样.

Read-after-Write依赖是有用的, 因为读取访问可能会读取由写入访问所写的值. 读后写依赖也叫作`anti-dependences`性. 它也是有用的, 因为写入访问可能会覆盖读取访问所读取的值. 写后写依赖也叫作`output dependences`, 也是有用的, 因为第二个写入访问可能会覆盖第一个写入访问所写的值, 在第二次之前执行要执行的第一次写入来确保将最终值写给原始程序中的数据元素的最终值不会出现错误. 而一对读取访问不会产生实际依赖, 因为这两个读取不会相互影响. 但是如果考虑为优化目的读取相同内存元素的一对语句实例, 仍然是有用的. 与真依赖性类似, 此类语句实例有时被称为读后读依赖或`input dependences`, 它与实际依赖的情况相反, 对于输入依赖, 两个语句实例的顺序并不重要.

接下来看一下例子:
```c
for (int i = 0; i < n; ++i) { 
  S: t = A[i]; 
  T: B[i] = t; 
}
```

statement`S`和`T`分别写入变量`t`和读取`t`. 


| |||
|-|-|-|
| 写后读依赖 |$${ S[i] \rightarrow T[i'] : i' \geq i }$$| ![](images/dep_a.png) |
| 读后写依赖 |$${ T[i] \rightarrow S[i'] : i' > i }$$| ![](images/dep_c.png) |
| 写后写依赖 |$${ S[i] \rightarrow S[i'] : i' > i }$$| ![](images/dep_d.png) |

到目前为止所描述的依赖的主要目的是确保在读取值之前将值写入内存, 并且之间没有被覆盖. 在某些情况下, 可能有很多这些依赖性, 以至于实例的执行顺序几乎无法更改甚至根本无法更改. 例如, 如果使用临时标量变量来存储不同的数据, 则上述依赖性将序列化访问该标量变量的语句实例. 通过将不同的数据存储在不同的内存位置中, 其中一些依赖性不再需要, 并且得到更大的自由度来更改语句实例的执行顺序. 有时一些写后读依赖和写后写依赖可以消除, 因此它们也被共同被称为`false dependences`


🟢 Dataflow Dependence Relation

  数据流依赖关系是(确切的)写后读依赖关系的子集, 包含没有对相同的数据进行写入的同一个语句实例.


数据流依赖性也称为`value-based dependences`, 因为该值沿依赖关系保留. 相反, 先前描述的依赖性也称为`memory-based dependences`的依赖性, 因为它们仅访问相同的内存位置.

和之前一样, 也可能无法确定或表示数据流依赖关系, 并且与写入访问一样, 重要的是要区分潜在的数据流和确定的数据流. 所以也有以下两种类型的数据流依赖关系:

- May-Dataflow Dependence Relation

  包含数据流依赖关系作为子集的二元关系.

- Must-Dataflow Dependence Relation

  数据流依赖关系的二元关系子集.

两者都以“标记”形式出现,每个语句实例都伴随着引用标识符,例如之前提到的`Tagged Access Relations`. 这些称为`tagged May-DataFlow`依赖关系和`tagged Must-Dataflow`依赖关系. 通常Must-Dataflow依赖关系是May-Dataflow依赖关系的子集. 如果可以准确执行数据流分析, 则两者彼此相等. May-DataFlow依赖关系本身就是(May)写后读依赖关系的子关系. 无标记的Must-Dataflow依赖关系仅在每个语句最多包含一个写入访问时才有用.


考虑之前的例子, 通过`S`的实例写入标量t的每个值都被同一语句的下一个实例所覆盖. 这意味着该值仅由语句T的一个中间实例读取. 也就是说，数据流依赖关系为:

| |||
|-|-|-|
| 数据流依赖 | $$ { S[i] \rightarrow T[i] } $$ | ![](images/dep_b.png) |

## Dependence Analysis

现在开始考虑依赖分析, 从访问关系和调度来计算依赖关系非常容易. 考虑计算写后读依赖关系的例子, 设$W$为May-Write访问关系, 设$R$为May-Read访问关系, 首先计算对于相同的元素一个执行写入, 一个执行读取的实例对, 可以通过将May-Write访问关系与May-Read访问关系的反向组合来获得此关系:

$$ R ^{-1} \circ W$$

然后, 保留第一个实例在第二个实例执行之前执行的实例对, 也就是说, 保留下来满足schedule order的实例:

$$(R ^{-1} \circ W)\ \cap < S$$

其他的读后写和写后写关系也可以使用类似的方法计算:

$$(W ^{-1} \circ R)\ \cap < S$$
$$(W ^{-1} \circ W)\ \cap < S$$


对依赖关系的上述表达的直接评估需要从调度S计算执行顺序关系$< S$, 由于它是在此调度中的成对域元素之间的关系，相应的范围的元素按词典序排列. 那么，执行命令关系就是与调度和自身之间词典序小于关系:
$$< S = \bf{S} ≺ \bf{S} $$


⚠️ : 这里$ B \circ A = { i \rightarrow j : ∃ k : i \rightarrow k \in A \land k \rightarrow j \in B }$, 在isl中为`A.apply_range(B)`.

In [9]:
scop = parse_code("""
float f1(float ); 
float f2(float ); 
void f(int n, float A[ restrict static n], float B[ restrict static n]) { 
#pragma scop  

  float t; 
  for (int i = 0; i < n; ++i) { 
S: t = f1(A[i]); 
T: B[i] = f2(t); 
  }
#pragma endscop   
}
""",'f')

In [17]:
sch =  scop.get_schedule()
root = sch.get_root()
order = sch.get_map() # get map用于获得schedule tree的lex order.
print("order",order)
order_relation = order.lex_lt_union_map(order)
print("order_relation", order_relation)
print("may_writes", scop.get_may_writes())
print("may_reads", scop.get_may_reads())
print("写与读进行composition", scop.get_may_writes().apply_range(scop.get_may_reads().reverse()))
print("写后读依赖", scop.get_may_writes().apply_range(scop.get_may_reads().reverse()).intersect(order_relation))

order [n] -> { T[i] -> [1, i, 1]; S_0[] -> [0, 0, 0]; S[i] -> [1, i, 0] }
order_relation [n] -> { S[i] -> T[i'] : i' > i; S[i] -> T[i' = i]; S[i] -> S[i'] : i' > i; S_0[] -> T[i]; S_0[] -> S[i]; T[i] -> T[i'] : i' > i; T[i] -> S[i'] : i' > i }
may_writes [n] -> { T[i] -> B[i] : 0 <= i < n; S[i] -> t[] : 0 <= i < n }
may_reads [n] -> { T[i] -> t[] : 0 <= i < n; S[i] -> A[i] : 0 <= i < n }
写与读进行composition [n] -> { S[i] -> T[i'] : 0 <= i < n and 0 <= i' < n }
写后读依赖 [n] -> { S[i] -> T[i'] : 0 <= i < n and i' > i and 0 <= i' < n; S[i] -> T[i' = i] : 0 <= i < n }


解释一下计算写后读依赖的过程, 首先`may_writes [n] -> { T[i] -> B[i] : 0 <= i < n; S[i] -> t[] : 0 <= i < n }`表示每个循环中`S`写入标量`t`, `T`写入数组`B`;`may_reads [n] -> { T[i] -> t[] : 0 <= i < n; S[i] -> A[i] : 0 <= i < n }`表示每个循环中`T`读取标量`t`, `S`读取数组`A`. 
将写与读的逆进行`compostion`之后, 也就是写出的目标元素和读入的目标元素进行匹配抵消, 获得写入实例和读入实例的读写关系`[n] -> { S[i] -> T[i'] : 0 <= i < n and 0 <= i' < n }`. 获取读写关系之后再消除不符合顺序关系的部分, 就得到了正确的写后读依赖`[n] -> { S[i] -> T[i'] : 0 <= i < n and i' > i and 0 <= i' < n; S[i] -> T[i' = i] : 0 <= i < n }`. 此时由S写入t, 由T读取t, 因此i与i'的范围相同, 且相等. 由S写入t, 下一次循环T读取t, 因此i与i'的范围相同, 且i’在i之后.

In [ ]:
print("读后写依赖", scop.get_may_reads().apply_range(scop.get_may_writes().reverse()).intersect(order_relation))
# 由T读取t, 下一个循环由S进行写入, 因此i与i'的范围相同, 且i’在i之后.
print("写后写依赖", scop.get_may_writes().apply_range(scop.get_may_writes().reverse()).intersect(order_relation))
# 由S写入t, 下一个循环再次写入, 因此i与i'的范围相同, 且i’在i之后.

## Dataflow Analysis

计算数据流依赖会更加复杂. 假设依赖分析已经被精确计算了, 这就表示所有的读写访问也被精确表示了. 接下来描述一些计算数据流依赖的方法以及各个方法的优缺点.

### killed by the intermediate write

上面说过, 数据流依赖性是一种没有对同一内存位置的中间写入的写后读依赖性. 计算数据流依赖性的一种方法是删除那些存在此类中间写入的写后读依赖性, 称为被中间写入`killed`. 为了能够将中间写入与正确的写后读依赖性相匹配, 依赖性需要跟踪所涉及的内存元素. 
因此读取访问关系$R$, 取得当前语句读取数据的关系, 在isl中表示为`range map`:

$$ R_1 = \xrightarrow[]{ran}\ R$$

同样, 获取写入访问关系$W$的写入数据关系:

$$ W_1 = \xrightarrow[]{ran} W$$

现在得到了写入和读取对于数据的关系, 然后需要将这些数据关系进行order, 所以需要合并读写关系并取domain, 再通过composition原始schedule获得当前访问数据关系domain的order:

$$ S_1 = S \circ (\xrightarrow[]{dom} (R \cup W)) $$

然后, 与相关数组元素的写后读依赖性参考上面的方法, 可以计算为:

$$ D_1 = (R^{-1}_1 \circ W_1) \cup <S_1 $$

类似的, 对于数组元素的写后写依赖性可以计算为:

$$ O_1 = (W^{-1}_1 \circ W_1) \cup <S_1 $$

然后, 数据流依赖性是写后读依赖, 除去写后写依赖性和写后读依赖性组合部分, 即:

$$ F_1 = D_1 \setminus (D_1 \circ O_1) $$

对数组元素的引用可以通过从$F_1$中删除domain factor来获得.


In [18]:
r1 = scop.get_may_reads().range_map()
print("r1:", r1)
w1 = scop.get_may_writes().range_map()
print("w1:", w1)
S = scop.get_schedule().get_map()
print("S:", S)
s1 = scop.get_may_reads().union(scop.get_may_writes()).domain_map().apply_range(S)
print("s1:", s1)
order_s1 = s1.lex_lt_union_map(s1)
print("order_s1:", order_s1)
d1 = w1.apply_range(r1.reverse()).intersect(order_s1)
print("写后读依赖:", d1)
o1 = w1.apply_range(w1.reverse()).intersect(order_s1)
print("写后写依赖:", o1)
f1 = d1.subtract(o1.apply_range(d1))
print("数据流依赖:", f1)

r1: [n] -> { [T[i] -> t[]] -> t[] : 0 <= i < n; [S[i] -> A[i]] -> A[i] : 0 <= i < n }
w1: [n] -> { [S[i] -> t[]] -> t[] : 0 <= i < n; [T[i] -> B[i]] -> B[i] : 0 <= i < n }
S: [n] -> { T[i] -> [1, i, 1]; S_0[] -> [0, 0, 0]; S[i] -> [1, i, 0] }
s1: [n] -> { [S[i] -> t[]] -> [1, i, 0] : 0 <= i < n; [T[i] -> t[]] -> [1, i, 1] : 0 <= i < n; [T[i] -> B[i]] -> [1, i, 1] : 0 <= i < n; [S[i] -> A[i]] -> [1, i, 0] : 0 <= i < n }
order_s1: [n] -> { [S[i] -> t[]] -> [S[i'] -> A[i']] : 0 <= i < n and i' > i and 0 <= i' < n; [T[i] -> B[i]] -> [T[i'] -> t[]] : 0 <= i < n and i' > i and 0 <= i' < n; [T[i] -> B[i]] -> [S[i'] -> A[i']] : 0 <= i < n and i' > i and 0 <= i' < n; [T[i] -> t[]] -> [T[i'] -> t[]] : 0 <= i < n and i' > i and 0 <= i' < n; [S[i] -> A[i]] -> [S[i'] -> A[i']] : 0 <= i < n and i' > i and 0 <= i' < n; [T[i] -> t[]] -> [T[i'] -> B[i']] : 0 <= i < n and i' > i and 0 <= i' < n; [S[i] -> t[]] -> [T[i'] -> t[]] : 0 <= i < n and i' > i and 0 <= i' < n; [S[i] -> t[]] -> [T[i' = i] -> t[]] 

上面这种计算数据流依赖性的方法的主要问题是它不能轻松地处理近似值, 特别是, D1 用于 (6.11) 中减法运算的两侧, 这意味着如果 D1 是过近似，则不能保证结果是过近似或欠近似，而大多数实际用途取决于这样的保证